
# Ejercicio 7: Bases de Datos Vectoriales

## Objetivo de la práctica

Entender el concepto de Bases de Datos Vectoriales y saber utilizar las herramientas actuales

## Parte 0: Carga del Corpus

Vamos a utilizar la API de Kaggle para acceder al dataset _Wikipedia Text Corpus for NLP and LLM Projects_

El corpus está disponible desde este [link](https://www.kaggle.com/datasets/gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects?utm_source=chatgpt.com)

### Actividad

1. Carga el corpus


In [1]:
import kagglehub
from kagglehub import KaggleDatasetAdapter

In [2]:
# Set the path to the file you'd like to load
file_path = "wikipedia_text_corpus.csv"

# Load the latest version
df = kagglehub.dataset_load(
  KaggleDatasetAdapter.PANDAS,
  "gzdekzlkaya/wikipedia-text-corpus-for-nlp-and-llm-projects",
  file_path,
)

df.head()

Using Colab cache for faster access to the 'wikipedia-text-corpus-for-nlp-and-llm-projects' dataset.


,Unnamed: 0,text
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...
1,2,Battery indicator\n\nA battery indicator (also...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...


## Parte 1: Generación de Embeddings

Vamos a utilizar E5 como modelo de embeddings.

La documentación de E5 está disponible desde este [link](https://huggingface.co/intfloat/e5-base-v2)

### Actividad

1. Normalizar el corpus
2. Definir una función `chunk_text`, y dividir los textos en _chunks_.
3. Generar embeddings por cada _chunk_

In [3]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
import re

df = df.dropna(subset=["text"]).reset_index(drop=True)

# Limpieza básica
def normalize_text(s: str) -> str:
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["text_norm"] = df["text"].astype(str).map(normalize_text)

df.head()

,Unnamed: 0,text,text_norm
0,1,Anovo\n\nAnovo (formerly A Novo) is a computer...,Anovo Anovo (formerly A Novo) is a computer se...
1,2,Battery indicator\n\nA battery indicator (also...,Battery indicator A battery indicator (also kn...
2,3,"Bob Pease\n\nRobert Allen Pease (August 22, 19...","Bob Pease Robert Allen Pease (August 22, 1940Â..."
3,4,CAVNET\n\nCAVNET was a secure military forum w...,CAVNET CAVNET was a secure military forum whic...
4,5,CLidar\n\nThe CLidar is a scientific instrumen...,CLidar The CLidar is a scientific instrument u...


In [ ]:
def chunk_text(text: str, max_chars: int = 800, overlap: int = 100):
    """
    Chunking por caracteres.
    max_chars ~ 600-1000 suele funcionar bien.
    overlap ayuda a no cortar ideas a la mitad.
    """
    chunks = []
    start = 0
    n = len(text)
    while start < n:
        end = min(start + max_chars, n)
        chunk = text[start:end]
        chunk = chunk.strip()
        if len(chunk) > 0:
            chunks.append(chunk)
        if end == n:
            break
        start = max(0, end - overlap)
    return chunks

records = []
for i, row in df.iterrows():
    chunks = chunk_text(row["text_norm"], max_chars=800, overlap=100)
    for j, ch in enumerate(chunks):
        records.append({
            "doc_id": int(i),
            "chunk_id": j,
            "text": ch
        })

chunks_df = pd.DataFrame(records)
chunks_df.head(), len(chunks_df)

In [ ]:
from sentence_transformers import SentenceTransformer

MODEL_NAME = "intfloat/e5-base-v2"   # recomendado para retrieval
model = SentenceTransformer(MODEL_NAME)

# Textos a indexar (pasajes)
passages = ["passage: " + t for t in chunks_df["text"].tolist()]

In [ ]:
# Embeddings (N x D)
# Se debe usar normalize_embeddings=True para similitud coseno
embeddings = model.encode(
    passages,
    batch_size=16,
    show_progress_bar=True,
    convert_to_numpy=True,
    normalize_embeddings=True
).astype("float32")

In [7]:
print(embeddings.shape, embeddings.dtype)

(79104, 768) float32


In [8]:
def embed_query(query: str) -> np.ndarray:
    q = "query: " + query
    vec = model.encode(
        [q],
        convert_to_numpy=True,
        normalize_embeddings=True
    ).astype("float32")
    return vec

query_text = "Battery measuring"

query_vec = embed_query(query_text)
query_vec.shape

(1, 768)

## Parte 2: FAISS

FAISS es una librería para búsqueda por similitud eficiente y clustering de vectores densos.

La documentación de FAISS está disponible en este [link](https://faiss.ai/index.html)

### Actividad

1. Crea un índice en FAISS
2. Carga los embeddings
3. Realiza una búsqueda a partir de una _query_

In [10]:
import torch
print(torch.version.cuda)

12.6


In [ ]:
!pip install faiss-cpu

In [13]:
import faiss
import numpy as np

# 1. Definir la consulta y generar su embedding
# Usando la funcion embed_query de antes
query_text = "What are the components of a battery?"
query_embedding = embed_query(query_text) # Genera el vector float32 de (1, D)

# 2. Configurar el indice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(embeddings)

# 3. Realizar la busqueda de los k=10 mas cercanos
k = 10
D, I = index.search(query_embedding, k=10)

# 4. Mostrar los resultados (ndices encontrados)
print(f"Busqueda finalizada para: '{query_text}'")
print("Indices chunks mas similares:", I[0])
print("Distancias L2 correspondientes:", D[0])

Busqueda finalizada para: 'What are the components of a battery?'
Indices chunks mas similares: [47064 27366     3 47068 10543 59382 61361 72557 23514 72563]
Distancias L2 correspondientes: [0.3330764  0.3478271  0.35039032 0.350646   0.350986   0.35197127
 0.35200125 0.35365903 0.3541758  0.35456187]


## Parte 3 — Vector DB #1: Qdrant (búsqueda vectorial + metadata)

### Objetivo
Recrear el mismo flujo que con FAISS, pero usando una base vectorial con soporte nativo de **metadata** y filtros.

### Qué debes implementar
1. Levantar / conectar con una instancia de Qdrant.
2. Crear una colección con:
   - dimensión `D` (la de tus embeddings)
   - métrica (cosine o L2)
3. Insertar:
   - `id`
   - `embedding`
   - `payload` (metadata: texto, título, etiquetas, etc.)
4. Consultar Top-k por similitud:
   - `query_embedding`
   - `k`

### Inputs esperados (ya definidos arriba en el notebook)
- `embeddings`: matriz `N x D` (float32)
- `texts`: lista de `N` strings
- `metadatas`: lista de `N` dicts (opcional)
- `query_text`: string
- `query_embedding`: vector `1 x D`

### Entregable
- Una función `qdrant_search(query_embedding, k)` que retorne:
  - lista de `(id, score, text, metadata)`
- Un ejemplo de consulta con `k=5` y su salida.



In [ ]:
!pip install qdrant-client

In [16]:
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

# Conectar en memoria
client = QdrantClient(":memory:")
COLLECTION_NAME = "wikipedia_collection"

In [17]:
# 1.Crear coleccion
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE),
)

# 2.Preparar los PointStruct para la insercion masiva
points = [
    PointStruct(
        id=idx,
        vector=vector.tolist(),
        payload={
            "text": chunks_df.iloc[idx]["text"],
            "doc_id": int(chunks_df.iloc[idx]["doc_id"]),
            "chunk_id": int(chunks_df.iloc[idx]["chunk_id"])
        }
    )
    for idx, vector in enumerate(embeddings)
]

# 3.Insertar en lotes (upsert)
client.upsert(collection_name=COLLECTION_NAME, points=points)
print(f"Colección '{COLLECTION_NAME}' lista con {len(points)} puntos.")

Colección 'wikipedia_collection' lista con 79104 puntos.


/tmp/ipython-input-606335975.py:22: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Current collection contains 79104 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client.upsert(collection_name=COLLECTION_NAME, points=points)


In [18]:
# Consulto Top-k por similitud
# Se ealiza la consulta y extrae los datos del payload de forma estructurada
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, PointStruct

In [19]:
# 1. Conexion limpia
client = QdrantClient(":memory:")
COLLECTION_NAME = "wiki_collection"

# 2. Recreacion e Insercion
if not client.collection_exists(COLLECTION_NAME):
    client.create_collection(
        collection_name=COLLECTION_NAME,
        vectors_config=VectorParams(size=embeddings.shape[1], distance=Distance.COSINE),
    )
# Preparar puntos para la T4
    points = [
        PointStruct(
            id=i,
            vector=embeddings[i].tolist(),
            payload={
                "text": chunks_df.iloc[i]["text"],
                "doc_id": int(chunks_df.iloc[i]["doc_id"]),
                "chunk_id": int(chunks_df.iloc[i]["chunk_id"])
            }
        )
        for i in range(len(embeddings))
    ]
    client.upsert(collection_name=COLLECTION_NAME, points=points)

# 3. Funcion de busqueda
def qdrant_search(query_embedding, k=5):
# Convertir el embedding de la query a lista plana
    query_vector = query_embedding.flatten().tolist()

# Con 'query_points' como alternativa robusta a search
    response = client.query_points(
        collection_name=COLLECTION_NAME,
        query=query_vector,
        limit=k,
        with_payload=True
    )

# Mapeo de resultados
    return [
        (point.id, point.score, point.payload["text"],
         {"doc_id": point.payload["doc_id"], "chunk_id": point.payload["chunk_id"]})
        for point in response.points
    ]

# 4. Ejemplo de consulta
query_text = "technology"
query_vec = embed_query(query_text)
resultados = qdrant_search(query_vec, k=5)

# 5. Salida formateada
print(f"Resultados: {query_text}\n" + "="*50)
for res in resultados:
    print(f"ID: {res[0]} | Score: {res[1]:.4f} | Metadata: {res[3]}")
    print(f"Texto: {res[2][:160]}...\n")

Resultados: technology
ID: 6013 | Score: 0.8411 | Metadata: {'doc_id': 828, 'chunk_id': 0}
Texto: Technology Technology ("science of craft", from Greek , "techne", "art, skill, cunning of hand"; and , "-logia") is the collection of techniques, skills, method...

ID: 52599 | Score: 0.8353 | Metadata: {'doc_id': 7230, 'chunk_id': 0}
Texto: Outline of technology The following outline is provided as an overview of and topical guide to technology: Technology â€“ collection of tools, including machine...

ID: 42677 | Score: 0.8251 | Metadata: {'doc_id': 5822, 'chunk_id': 6}
Texto: rmâ€™s technology is used in a variety of consumer electronics, including mobile phones and tablets, modems, gaming consoles, digital televisions, automotive sy...

ID: 68214 | Score: 0.8242 | Metadata: {'doc_id': 9300, 'chunk_id': 4}
Texto: Journal with a Technology Merit Award in the category of remediation for the invention of MAGS technology....

ID: 66500 | Score: 0.8226 | Metadata: {'doc_id': 9104, 'chunk_id'

### Preguntas
- ¿La métrica usada fue cosine o L2? ¿Por qué?
Se uso la métrica Coseno ya que en un espacio vectorial normalizado donde la dirección del vector es más relevante que su magnitud para determinar la similitud
- ¿Qué tan fácil fue filtrar por metadata en comparación con FAISS?
A diferencia de FAIS, aqui se intrgra la metadata directamente en el payload del vector, permitiendo la búsqueda y la recuperación de información se realicen en una única operación
- ¿Qué pasa con el tiempo de respuesta cuando aumentas `k`?
El tiempo de respuesta aumenta de forma marginal o despreciable para valores de k

## Parte 4 — Vector DB #2: Milvus (indexación ANN y escalabilidad)

### Objetivo
Implementar el flujo de indexación + búsqueda con una base vectorial orientada a escalabilidad.

### Qué debes implementar
1. Conectar a Milvus.
2. Crear un esquema (colección) con:
   - campo `id` (entero o string)
   - campo `embedding` (vector `D`)
   - campos de metadata (p.ej., `category`, `source`, `title`)
3. Insertar `N` embeddings.
4. Crear/seleccionar un índice ANN (ej. HNSW o IVF).
5. Ejecutar consultas Top-k y recuperar textos asociados.

### Recomendación didáctica
Haz dos configuraciones:
- **Búsqueda exacta** (si aplica) o configuración “más precisa”
- **Búsqueda ANN** (configuración “más rápida”)

Luego compara:
- tiempo de consulta
- overlap de resultados (cuántos IDs coinciden)

### Entregable
- Función `milvus_search(query_embedding, k)` que devuelva resultados.
- Un mini experimento: `k=5` y `k=20` (tiempos y resultados).


In [ ]:
!pip install "pymilvus[milvus_lite]"

In [21]:
from pymilvus import MilvusClient, DataType
import time

client_milvus = MilvusClient("milvus_wikipedia.db")
COLLECTION_NAME = "wiki_vectors"

In [22]:
import os
import signal
import gc

if 'client_milvus' in locals():
    del client_milvus

# Forzar recolección de basura
gc.collect()

# Borrar archivos de socket y base de datos
!rm -rf /tmp/milvus* !rm -f milvus_wikipedia.db*

print("Entorno de Milvus limpiado")

Entorno de Milvus limpiado


In [23]:
from pymilvus import MilvusClient
import time

# Nueva conexion con otro nombre
client_milvus = MilvusClient("milvus_wikipedia_v2.db")
COLLECTION_NAME = "wiki_vectors_fixed"

# Crear coleccion
client_milvus.create_collection(
    collection_name=COLLECTION_NAME,
    dimension=embeddings.shape[1],
    metric_type="COSINE"
)

In [24]:
# Insertar una muestra controlada
limite = 10000 # limite establecido
data_batch = [
    {
        "id": i,
        "vector": embeddings[i].tolist(),
        "text": chunks_df.iloc[i]["text"],
        "doc_id": int(chunks_df.iloc[i]["doc_id"])
    }
    for i in range(limite)
]

client_milvus.insert(collection_name=COLLECTION_NAME, data=data_batch)
print(f"{len(data_batch)} vectores insertados con éxito.")

10000 vectores insertados con éxito.


In [25]:
# Busqueda Exacta (FLAT)
index_params_exact = client_milvus.prepare_index_params()
index_params_exact.add_index(
    field_name="vector",
    index_type="FLAT",
    metric_type="COSINE"
)
client_milvus.create_index(COLLECTION_NAME, index_params_exact)
# Busqueda ANN (IVF_FLAT)
index_params_ann = client_milvus.prepare_index_params()
index_params_ann.add_index(
    field_name="vector",
    index_type="IVF_FLAT",
    metric_type="COSINE",
    params={"nlist": 128} # nlist es el # de clusters para la busqueda
)
client_milvus.create_index(COLLECTION_NAME, index_params_ann)

# Cargar la coleccion para habilitar la busqueda
client_milvus.load_collection(COLLECTION_NAME)

In [26]:
# Funcion de busqueda
def milvus_search(query_embedding, k=5):
    q_vec = query_embedding.flatten().tolist()
    start = time.time()
    res = client_milvus.search(
        collection_name=COLLECTION_NAME,
        data=[q_vec],
        limit=k,
        output_fields=["text", "doc_id"],
        search_params={"metric_type": "COSINE", "params": {"nprobe": 10}}
    )
    return res[0], (time.time() - start)

In [27]:
# Pruebas
query_text = "technology"
query_vec = embed_query(query_text)

print(f"Resultados IVF_FLAT: '{query_text}'")
for val_k in [5, 20]:
    hits, latencia = milvus_search(query_vec, k=val_k)
    print(f"\n[k={val_k}] Latencia: {latencia:.6f}s")
    print(f"Mejor ID: {hits[0]['id']} | Texto: {hits[0]['entity']['text'][:80]}...")

Resultados IVF_FLAT: 'technology'

[k=5] Latencia: 0.012442s
Mejor ID: 6013 | Texto: Technology Technology ("science of craft", from Greek , "techne", "art, skill, c...

[k=20] Latencia: 0.006600s
Mejor ID: 6013 | Texto: Technology Technology ("science of craft", from Greek , "techne", "art, skill, c...


### Preguntas
- ¿Qué parámetros del índice/control de búsqueda ajustaste para precisión vs velocidad?
 se definió nlist=128. Este parámetro determina en cuántos "clústeres" o regiones se divide el espacio vectorial. Para el de búsqueda se utilizó nprobe=10 en los search_params, como el control crítico que define cuántos de esos clústeres se revisarán durante la consulta; que para nprobe cercano a 1 siendo ultra rápido
- ¿Qué evidencia tienes de que ANN cambia los resultados (aunque sea poco)?
La evidencia más clara se obtiene al comparar la lista de IDs devuelta por el índice FLAT  contra la de IVF_FLAT


## Parte 5 — Vector DB #3: Weaviate (búsqueda semántica con esquema)

### Objetivo
Montar una colección con esquema (clase) y ejecutar búsquedas semánticas Top-k, opcionalmente con filtros.

### Qué debes implementar
1. Conectar a Weaviate.
2. Definir un esquema:
   - Clase/colección (por ejemplo `Document`)
   - Propiedades: `text`, `title`, `category`, etc.
   - Vector asociado (embedding)
3. Insertar objetos con:
   - propiedades + vector
4. Consultar por similitud (Top-k) con `query_embedding`.
5. (Opcional) agregar un filtro por propiedad (metadata).

### Recomendación
Asegúrate de guardar el `text` original y al menos 1 campo de metadata para probar filtrado.

### Entregable
- Función `weaviate_search(query_embedding, k)` que retorne:
  - id, score, text, metadata



In [ ]:
!pip install -U weaviate-client

In [ ]:
import weaviate
import os

# Conexin a instancia embebida
client_weaviate = weaviate.connect_to_embedded()
print("Conexion exitosa a Weaviate Embebido!")

In [30]:
import weaviate.classes.config as wvcc

# Definir nombre de la coleccion
COLLECTION_NAME = "Document"

# Eliminar coleccion si ya existia
if client_weaviate.collections.exists(COLLECTION_NAME):
    client_weaviate.collections.delete(COLLECTION_NAME)

# Crear la coleccin con sus propiedades
client_weaviate.collections.create(
    name=COLLECTION_NAME,
    vectorizer_config=None,
    properties=[
        wvcc.Property(name="text", data_type=wvcc.DataType.TEXT),
        wvcc.Property(name="doc_id", data_type=wvcc.DataType.INT),
        wvcc.Property(name="chunk_id", data_type=wvcc.DataType.INT),
    ]
)
print(f"Esquema definido: '{COLLECTION_NAME}")

Esquema definido: 'Document


In [31]:
# Obtener referencia a la coleccion creada en el paso anterior
documents = client_weaviate.collections.get("Document")

# Configurar la inserción por lotes (Batch)
limite_insercion = 10000

print(f"Insertando {limite_insercion} objetos en Weaviate...")

with documents.batch.dynamic() as batch:
    for i in range(limite_insercion):
        batch.add_object(
            properties={
                "text": chunks_df.iloc[i]["text"],
                "doc_id": int(chunks_df.iloc[i]["doc_id"]),
                "chunk_id": int(chunks_df.iloc[i]["chunk_id"])
            },
            vector=embeddings[i].tolist()
        )

# Validar la carga
# En Weaviate v4, podemos usar aggregate para contar los objetos
total = documents.aggregate.over_all(total_count=True)
print(f"Éxito: Se han insertado {total.total_count} objetos en la colección.")

Insertando 10000 objetos en Weaviate...
Éxito: Se han insertado 10000 objetos en la colección.


In [32]:
# Definir la query y generar el embedding (usando tu función de la Parte 1)
query_text = "How does a lead-acid battery store energy?"
query_vec = embed_query(query_text)

# Realizar la busqueda por similitud vectorial
# Weaviate requiere que el vector sea una lista plana
results = documents.query.near_vector(
    near_vector=query_vec.flatten().tolist(),
    limit=5,
    return_metadata=weaviate.classes.query.MetadataQuery(distance=True)
)

# Mostrar resultados
print(f"Resultados para: '{query_text}'\n")
for i, obj in enumerate(results.objects):
    distancia = obj.metadata.distance
    texto = obj.properties['text']
    meta = f"Doc: {obj.properties['doc_id']}, Chunk: {obj.properties['chunk_id']}"

    print(f"{i+1}. [Distancia: {distancia:.4f}]")
    print(f"   Metadata: {meta}")
    print(f"   Contenido: {texto[:150]}...\n")

Resultados para: 'How does a lead-acid battery store energy?'

1. [Distancia: 0.1701]
   Metadata: Doc: 993, Chunk: 19
   Contenido: eded. This DC power is supplied by the battery pack, and the controller regulates the power to the motor, supplying either variable pulse width DC or ...

2. [Distancia: 0.1771]
   Metadata: Doc: 1123, Chunk: 5
   Contenido: battery. Heat pumps with hot water storage and electric vehicles have been found to have higher potential on reduction of emissions and fossil fuel us...

3. [Distancia: 0.1786]
   Metadata: Doc: 909, Chunk: 0
   Contenido: Gravity battery A gravity battery is a type of mechanical battery that stores gravitational potential energy, by raising a mass, allowing that energy ...

4. [Distancia: 0.1862]
   Metadata: Doc: 1171, Chunk: 1
   Contenido: the anode. This eliminates the intermediate step of producing hydrogen through the costly reforming process. Gaseous molecules of the hydrocarbon fuel...

5. [Distancia: 0.1867]
   Metadata: Do

In [34]:
# Funcion entregable
import weaviate.classes.query as wvq

def weaviate_search(query_embedding, k=5):
    # Convertir el embedding a lista plana
    query_vector = query_embedding.flatten().tolist()

    # Ejecutar consulta vectorial
    response = documents.query.near_vector(
        near_vector=query_vector,
        limit=k,
        return_metadata=wvq.MetadataQuery(distance=True)
    )

    formatted_results = []

    for obj in response.objects:
        # Extraer datos requeridos
        res_id = obj.uuid
        score = obj.metadata.distance
        text = obj.properties.get("text")

        # Agrupar el resto de la metadata
        metadata = {
            "doc_id": obj.properties.get("doc_id"),
            "chunk_id": obj.properties.get("chunk_id")
        }

        formatted_results.append((res_id, score, text, metadata))

    return formatted_results

# Ejemplo de ejecucion
query_text = "What is the history of batteries?"
query_vec = embed_query(query_text)
resultados = weaviate_search(query_vec, k=5)

print(f"Resultados:\n" + "================")
for res in resultados:
    uid, dist, txt, meta = res
    print(f"ID: {uid}")
    print(f"Score (Distancia): {dist:.4f}")
    print(f"Metadata: {meta}")
    print(f"Texto: {txt[:120]}...\n" + "---------------")

Resultados:
ID: 84d055ac-c53e-43d3-be6a-1d592860341e
Score (Distancia): 0.1983
Metadata: {'doc_id': 909, 'chunk_id': 0}
Texto: Gravity battery A gravity battery is a type of mechanical battery that stores gravitational potential energy, by raising...
---------------
ID: 3fe8c7c2-d410-46b0-b8cd-f33ff5eafb30
Score (Distancia): 0.2043
Metadata: {'doc_id': 867, 'chunk_id': 18}
Texto: lity. Where the mines were lucrative and their energy consumption high as a result of shaft depth or the ingress of wate...
---------------
ID: cd5408b5-8b72-439f-84dd-cfaccca1c4fe
Score (Distancia): 0.2066
Metadata: {'doc_id': 430, 'chunk_id': 0}
Texto: Power history Power History refers to the power of a nuclear reactor over an extended period of time. Power history is i...
---------------
ID: 3f7ebf01-67ed-4dee-950b-888c172d4ae7
Score (Distancia): 0.2083
Metadata: {'doc_id': 993, 'chunk_id': 20}
Texto: ry technology for EVs has developed from early lead-acid batteries used in the late 19th Century to the 20

### Preguntas
- ¿Qué diferencia conceptual encuentras entre “schema + objetos” vs “tabla + filas”?
La "tabla + filas" Milvus/FAISS es un modelo relacional/plano donde los datos son registros en una estructura rígida, mientras el modelo de "schema + objetos" de Weaviate es orientado a entidades lo cual permite definir clases que representan conceptos del mundo real con varios tipos de datos
- ¿Cómo describirías el trade-off de complejidad vs expresividad?Requiere una mayor configuración inicial, ya que hay que definir tipos de datos, clases y configuraciones de vectorización

## Parte 6 — Vector Store #4: Chroma (prototipado rápido)

### Objetivo
Implementar la misma idea de indexación y búsqueda semántica con una herramienta ligera de prototipado.

### Qué debes implementar
1. Crear una colección.
2. Insertar:
   - ids
   - embeddings
   - documents (texto)
   - metadatas (opcional)
3. Consultar Top-k con `query_embedding`.

### Nota didáctica
Chroma es útil para prototipos: enfócate en reproducir el pipeline sin “infra pesada”.

### Entregable
- Función `chroma_search(query_embedding, k)` que retorne resultados.
- Una consulta con `k=5`.



In [ ]:
!pip install chromadb

In [37]:
import chromadb
import time

chroma_client = chromadb.Client()
collection_chroma = chroma_client.create_collection(name="wiki_collection")

In [38]:
# Preparar las listas de datos
all_embeddings = embeddings.tolist()
all_documents = chunks_df['text'].tolist()
all_metadatas = [{"doc_id": int(d), "chunk_id": int(c)}
                 for d, c in zip(chunks_df['doc_id'], chunks_df['chunk_id'])]
all_ids = [str(i) for i in range(len(chunks_df))]

# Configurar el tamanio del lote
batch_size = 5000
total_len = len(all_ids)

print(f"Iniciando inserción de {total_len} documentos en lotes de {batch_size}...")

# Bucle de insercion por lotes
for i in range(0, total_len, batch_size):
    end_idx = min(i + batch_size, total_len)

    collection_chroma.add(
        embeddings=all_embeddings[i:end_idx],
        documents=all_documents[i:end_idx],
        metadatas=all_metadatas[i:end_idx],
        ids=all_ids[i:end_idx]
    )
    print(f"Progreso: {end_idx}/{total_len} documentos insertados")

print(f"\nExito final: {collection_chroma.count()} documentos indexados en Chroma")


Iniciando inserción de 79104 documentos en lotes de 5000...
Progreso: 5000/79104 documentos insertados
Progreso: 10000/79104 documentos insertados
Progreso: 15000/79104 documentos insertados
Progreso: 20000/79104 documentos insertados
Progreso: 25000/79104 documentos insertados
Progreso: 30000/79104 documentos insertados
Progreso: 35000/79104 documentos insertados
Progreso: 40000/79104 documentos insertados
Progreso: 45000/79104 documentos insertados
Progreso: 50000/79104 documentos insertados
Progreso: 55000/79104 documentos insertados
Progreso: 60000/79104 documentos insertados
Progreso: 65000/79104 documentos insertados
Progreso: 70000/79104 documentos insertados
Progreso: 75000/79104 documentos insertados
Progreso: 79104/79104 documentos insertados

Exito final: 79104 documentos indexados en Chroma


In [39]:
def chroma_search(query_embedding, k=5):
    # Chroma permite pasar el vector directamente como lista
    query_vector = query_embedding.flatten().tolist()

    start_time = time.time()
    results = collection_chroma.query(
        query_embeddings=[query_vector],
        n_results=k,
        include=["documents", "metadatas", "distances"]
    )
    latency = time.time() - start_time

    # Formatear salida para el usuario
    formatted_results = []
    for i in range(len(results['ids'][0])):
        formatted_results.append({
            "id": results['ids'][0][i],
            "score": results['distances'][0][i],
            "text": results['documents'][0][i],
            "metadata": results['metadatas'][0][i]
        })

    return formatted_results, latency

# Ejecucion
query_text = "how much lead is in a typical battery?"
query_vec = embed_query(query_text)

res_chroma, t_chroma = chroma_search(query_vec, k=5)

print(f"Resultados de Chroma (Latencia: {t_chroma:.4f}s) para: '{query_text}'\n" + "-"*80)
for r in res_chroma:
    print(f"ID: {r['id']} | Distancia: {r['score']:.4f}")
    print(f"Meta: {r['metadata']}")
    print(f"Texto: {r['text'][:120]}...\n")

Resultados de Chroma (Latencia: 0.0747s) para: 'how much lead is in a typical battery?'
--------------------------------------------------------------------------------
ID: 46415 | Distancia: 0.2676
Meta: {'doc_id': 6358, 'chunk_id': 36}
Texto: cies ranged from 91% to 94.5%, depending on the batteryâ€™s state of charge. [REF] This is compared with a Sandia Nation...

ID: 71872 | Distancia: 0.2928
Meta: {'chunk_id': 2, 'doc_id': 9888}
Texto: is achieved. Accepted average float voltages for lead-acid batteries at 25 Â°C can be found in following table: Compensa...

ID: 72562 | Distancia: 0.3025
Meta: {'chunk_id': 5, 'doc_id': 9996}
Texto: dmium) and NiMH (nickel metal hydride) typically output 1.25Â volts per cell. Devices intended for use with primary batt...

ID: 72559 | Distancia: 0.3088
Meta: {'doc_id': 9996, 'chunk_id': 2}
Texto: tituting a battery. The current IEC standards for portable primary (non-rechargeable) batteries bear the 60086 number. T...

ID: 59383 | Distancia: 0.3088


### Preguntas
- ¿Qué tan fácil fue implementar todo comparado con Qdrant/Milvus?
 Chroma permite una integración casi plana y no hubo necesidad de configurar parámetros complejos de índices para que funcionara de inmediato
- ¿Qué limitaciones ves para un sistema en producción?
Para mi no gestiona tan bien los millones de vectores como lo hace la arquitectura distribuida de Milvus

## Parte 7 — SQL + vectores: PostgreSQL/pgvector (vector search transparente)

### Objetivo
Guardar embeddings en una tabla y ejecutar una consulta SQL de similitud.

### Qué debes implementar
1. Conectar a una base PostgreSQL con `pgvector` habilitado.
2. Crear una tabla (ej. `documents`) con:
   - `id` (PK)
   - `text` (texto)
   - `embedding` (vector(D))
   - metadata (columnas adicionales)
3. Insertar todos los documentos y embeddings.
4. Consultar Top-k por similitud, ordenando por distancia.

### Fórmula conceptual (lo que implementa tu SQL)
Para una consulta `q`, buscas:
$$ argmin_d \in D \; \text{dist}(\vec{q}, \vec{d})$$
donde `dist` puede ser L2 o una variante para cosine (según configuración).

### Entregable
- Función `pgvector_search(query_embedding, k)` que ejecute SQL y devuelva:
  - id, score/distancia, text, metadata




In [ ]:
!apt-get update
!apt-get -y install postgresql-server-dev-14 build-essential

In [ ]:
# Clonar y compilar pgvector desde el codigo fuente
!git clone https://github.com/pgvector/pgvector.git
%cd pgvector
!make
!make install
%cd ..

# Reiniciar el servicio de Postgres
!service postgresql restart

In [ ]:
# Instalacion y Configuración
!apt-get -y install postgresql-14
!service postgresql start
!sudo -u postgres psql -c "CREATE USER colab WITH PASSWORD 'colab';"
!sudo -u postgres psql -c "CREATE DATABASE vector_db OWNER colab;"
!sudo -u postgres psql -d vector_db -c "CREATE EXTENSION IF NOT EXISTS vector;"

# Librería de Python para interactuar con pgvector
!pip install pgvector psycopg2-binary

In [ ]:
# Elevar al usuario colab a sudo
!sudo -u postgres psql -c "ALTER USER colab WITH SUPERUSER;"

print("Privilegios de sudo a 'colab'.")

In [44]:
import psycopg2
from pgvector.psycopg2 import register_vector

# Conexion
conn = psycopg2.connect(
    dbname="vector_db",
    user="colab",
    password="colab",
    host="localhost"
)
cur = conn.cursor()

# Activacion
cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
conn.commit()

# Registro del tipo en Python
register_vector(conn)

# Creacion de la tabla
dimension = embeddings.shape[1]
cur.execute("DROP TABLE IF EXISTS documents;")
cur.execute(f"""
    CREATE TABLE documents (
        id SERIAL PRIMARY KEY,
        text TEXT,
        doc_id INTEGER,
        chunk_id INTEGER,
        embedding vector({dimension})
    );
""")
conn.commit()

print(f"Tabla creada con exito")

Tabla creada con exito


In [45]:
# Preparar los datos en el formato que espera la consulta SQL como array
print("Preparando datos para la inserción...")
data_to_insert = [
    (
        chunks_df.iloc[i]["text"],
        int(chunks_df.iloc[i]["doc_id"]),
        int(chunks_df.iloc[i]["chunk_id"]),
        embeddings[i].tolist()
    )
    for i in range(len(embeddings))
]

# Definir consulta SQL
insert_query = """
    INSERT INTO documents (text, doc_id, chunk_id, embedding)
    VALUES (%s, %s, %s, %s);
"""

# Ejecutar la insercion por lotes
batch_size = 2000
print(f"Insertando {len(data_to_insert)} registros en PostgreSQL...")

try:
    execute_batch(cur, insert_query, data_to_insert, page_size=batch_size)
    conn.commit()
    print("Inserción masiva completada exitosamente.")
except Exception as e:
    conn.rollback()
    print(f"Error durante la inserción: {e}")

# Verificar el conteo final
cur.execute("SELECT COUNT(*) FROM documents;")
count = cur.fetchone()[0]
print(f"Total de filas en la tabla 'documents': {count}")

Preparando datos para la inserción...
Insertando 79104 registros en PostgreSQL...
Error durante la inserción: name 'execute_batch' is not defined
Total de filas en la tabla 'documents': 0


In [46]:
conn.rollback()

In [50]:
#Realiza una búsqueda semantica con cast explícito para evitar errores de tipo
def pgvector_search(query_embedding, k=5):

    query_vec = query_embedding.flatten().tolist()
    search_query = """
    SELECT
        id,
        embedding <=> %s::vector AS score,
        text,
        doc_id,
        chunk_id
    FROM documents
    ORDER BY score ASC
    LIMIT %s;
    """

    try:
        cur.execute(search_query, (query_vec, k))
        rows = cur.fetchall()

        formatted_results = []
        for row in rows:
            formatted_results.append({
                "id": row[0],
                "score": row[1],
                "text": row[2],
                "metadata": {"doc_id": row[3], "chunk_id": row[4]}
            })
        return formatted_results
    except Exception as e:
        conn.rollback()
        return []

# Prueba
query_text = "how lead acid batteries are recycled"
query_vec = embed_query(query_text)
resultados_pg = pgvector_search(query_vec, k=5)

if resultados_pg:
    print(f"Busqueda exitosa en PostgreSQL:\n" + "=======")
    for res in resultados_pg:
        print(f"Score: {res['score']:.4f} | Texto: {res['text'][:100]}...")

### Preguntas
- ¿Qué tan “explicable” te parece esta aproximación vs las otras?
Es la más explicable de todas porque en pgvector la búsqueda es código SQL estándar
- ¿Qué ventajas ofrece el mundo SQL (JOIN, filtros, agregaciones)?
Con la experiencia que manejo un poco las consultas SQL,  como cruzar tus vectores con tablas de usuarios, ventas o logs históricos de forma nativa, o usar filtros complejos para combinar la búsqueda semántica con condiciones relacionales, me resulto un poco mas facil que lo anterior
- ¿Qué limitaciones esperas en escalabilidad frente a bases vectoriales dedicadas?
Yo diria el consumo de ram